# 写给自然语言处理小白——某NLP小白
同样作为一个NLP的新手，本系列将给大家分享简单实用的NLP必备知识和Python使用技巧，助大家轻松入门~

很快你将会学会如下操作，成为别人眼中的大佬。

↓↓↓

In [ ]:
import torch
import torch.nn as nn
import torch.nn.function as F
import torchtext
import pandas as pd
import jieba
import numpy as np

# NLP初步：数据预处理
本篇将介绍给大家NLP或数据科学工作当中的第一步，数据预处理，也就是如何读写你手头的数据，通常我们会见到的数据格式有`.json`/`.csv`/`.tsv`等。以下你将会学习：
* csv
* json
* yaml
* pandas
* sqlite

## csv/tsv
csv/tsv是最常见的保存数据的格式，Kaggle上绝大部分数据集都是csv。而Python里带有csv的第三方库，用于读写csv类的文件。

注意：csv也可以用pandas读，准确地说pandas啥都能读。。。但如果你只是单纯地想对行进行操作，pandas会麻烦那么一点，用csv就很简单粗暴，以下先介绍csv。

**读取**

In [2]:
import csv

file_name = 'csv_file0.csv'
with open(file_name, 'r') as f:
    file = csv.reader(f)

使用csv对文件操作首先需要调用open打开文件，随后定义一个读的对象。

这个类里的数据默认是按行进行存储的，因此接下来我们逐行来读取文件内容，也是最常用的操作。

假设文件每行长成这样，我们想要读取每行的第二列：
```csv
natural language processing
processing natural language
```

In [3]:
csv_file0 = 'csv_file0.csv'
raw_data = []

with open(csv_file0, 'r', encoding='utf-8') as f:
    file = csv.reader(f)
    for line in file:
        raw_data.append(line[1])

print(raw_data)

注意：encoding参数最好加上，不然默认会以gbk打开，很可能会是乱码。如果utf-8打开还是乱码的话可以使用utf-8-sig。

**写入**

In [5]:
csv_file1 = 'csv_file1.csv'
line = [1, 2, 3]
lines = [[1, 2, 3], [4, 5, 6]]

with open(csv_file1, 'w', encoding='utf-8', newline='') as f:
    file = csv.writer(f)
#     file.writerow(line)
    file.writerows(lines)

writerow接收一个一维数组，写入文件当前的一行，而writerows接收一个二维数组，会一次性写入多行。

注意：这里newline参数没有的话，会默认写完一行空一行，也就是两行文字之间会隔一行。另外这里的encoding会保证之后用utf-8肯定能打得开。

**tsv**

接下来讲一下tsv，和csv很像，区别是它以\t分隔，而csv以逗号分隔，因此只需要在csv的基础上指定一个分隔符参数即可。

In [9]:
tsv_file0 = 'tsv_file0.tsv'
raw_data = []

with open(tsv_file0, 'r', encoding='utf-8') as f:
    file = csv.reader(f, delimiter='\t')
    for line in file:
        raw_data.append(line[1])

tsv_file1 = 'tsv_file1.tsv'

with open(tsv_file1, 'w', encoding='utf-8', newline='') as f:
    file = csv.writer(f, delimiter='\t')
    file.writerows(lines)

## pandas
pandas的话就比较强大了，兼容许多数据格式，但相应操作也比较复杂。pandas打开的文件数据在内部以DataFrame的方式存在，有点像关系型数据库，以下尽量避开高端操作。

**读取csv/tsv**

In [15]:
import pandas as pd

raw_data = pd.read_csv(csv_file0)
for i in raw_data.index.values:
    print(raw_data.iloc[i, 3])
#     print(raw_data.loc[i, 'Age'])
    
raw_data = pd.read_csv(tsv_file0, sep='\t')

用pandas不需要事先调用open。

调用iloc会定位到第i行j列，而loc会根据列的名称来定位，假设你的文件长这样：
```csv
Id Name Gender Age
00 ginga male 20
```
那么上述两个写法的效果是一样。

**写入csv/tsv**

In [19]:
raw_data.to_csv(tsv_file1, sep='\t')

注意：调用to_xxx(file_name)时要保证xxx和file_name的格式一致。如果不想写入表头数据的话可以加header=0。

**excel**

有些时候数据是excel格式的，pandas也可以搞定，但需要安装依赖库xlrd。
```bash
conda install xlrd
```

In [ ]:
excel_file = 'excel_file.xlsx'

raw_data = pd.read_excel(excel_file)

## json/yaml
json和yaml都是存储字典数据的常用格式，Python当中也有相应的第三方库。

**json**

In [24]:
import json

json_file0 = 'json_file0.json'
json_file1 = 'json_file1.json'
data = {'a': '欸', 'b': '笔', 'c': '吸'}

with open(json_file0, 'r', encoding='utf-8') as f:
    raw_data = json.load(f)
    
with open(json_file1, 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

注意：indent参数最好加，不然生成的json文件会长得很丑，如果你的数据有中文，需声明ensure_ascii=False，不然文件里只会显示ascii码。

json除了上面的load和dump，你还会见到一种loads和dumps的用法，区别是前者接收的是文件对象，后者是字符串，所以要操作文件的话必须先read。

In [28]:
with open(json_file0, 'r', encoding='utf-8') as f:
    f_string = f.read()
    raw_data = json.loads(f_string)

with open(json_file1, 'w', encoding='utf-8') as f:
    f_string = json.dumps(data, indent=4, ensure_ascii=False)
    f.write(f_string)

**yaml**
```bash
pip install pyyaml
```
yaml与json的操作几乎一样，但yaml文件以缩进表示字典元素间的层级关系，-表示数组，并且文件里字符串不显示引号。
```yaml
name: ginga
age: 20
friends:
  name: mary
  age: 21
hobby:
  - sports
  - coding
```

In [35]:
import yaml

yaml_file0 = 'yaml_file0.yaml'
yaml_file1 = 'yaml_file1.yaml'

with open(yaml_file0, 'r', encoding='utf-8') as f:
    raw_data = yaml.load(f)
    
with open(yaml_file1, 'w', encoding='utf-8') as f:
    yaml.dump(data, f, allow_unicode=True)

注意：要显示中文的话同样需要声明allow_unicode=True。
## sqlite
有些时候数据是以数据库的形式存在的，例如`.db`文件。这时候可以用嵌入在Python当中的sqlite3数据库来进行操作。

In [34]:
import sqlite3

conn = sqlite3.connect("test.db")

cursor = conn.cursor()

sql = 'SELECT * FROM Student WHERE age=20'
cursor.execute(sql)

使用sqlite有如下几步：
* 建立数据库连接
* 创建游标对象
* 指定sql操作
* 执行sql操作

注意：如果sql语句中包含参数，需要将参数一起传入execute。

接下来就是考验大家MySQL的本领了~

以上就是第一期内容，相信你很快就能熟练运用Python处理数据了，剩下的就是跑模型了，换句话说你已经掌握了一半的NLP了！

## <font color=red>下回：教你如何使用BERT</font>